In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

import torch
import numpy as np
import random


import torch.nn as nn
from torch.utils.data import DataLoader

from helper_functions import *
from model_component import *
from parameters import *

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/home/dilan/anaconda3/envs/text_tag_combined_exp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = PuzzleImageDataset(img_dir='./dataset/test', data_file=None, label_file_name=None, grid_size=6)
dataset.preprocess(device, data_out_name="private_test_dataset")

test_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

Preprocessing of data started!
10 completed.
20 completed.
30 completed.
40 completed.
50 completed.
60 completed.
70 completed.
80 completed.
90 completed.
100 completed.
110 completed.
120 completed.
130 completed.
140 completed.
150 completed.
160 completed.
170 completed.
180 completed.
190 completed.
200 completed.
210 completed.
220 completed.
230 completed.
240 completed.
250 completed.
260 completed.
270 completed.
280 completed.
290 completed.
300 completed.
310 completed.
320 completed.
330 completed.
340 completed.
350 completed.
360 completed.
370 completed.
380 completed.
390 completed.
400 completed.
410 completed.
420 completed.
430 completed.
440 completed.
450 completed.
460 completed.
470 completed.
480 completed.
490 completed.
500 completed.
510 completed.
520 completed.
530 completed.
540 completed.
550 completed.
560 completed.
570 completed.
580 completed.
590 completed.
600 completed.
610 completed.
620 completed.
630 completed.
640 completed.
650 completed.
660

In [4]:
from helper_functions import *
from model_component import *
from parameters import *
from os import listdir
from os.path import isfile, join

models_dir = "ensemble_prediction"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_files = [f for f in listdir(models_dir) if (isfile(join(models_dir, f)) and f.endswith('cpt'))]
models = []

for model_name in model_files:

    model = PermutationPredictor(num_heads=NUM_HEADS, dropout=DROP_OUT)
    model.to(device)
    model.load_state_dict(torch.load(join(models_dir, model_name)))
    model.eval()

    models.append(model)

len(models)

20

In [5]:
true_lbls = []
predicted_lbls = []
for i, data in enumerate(iter(test_dataloader), 0):
    
    img_vecs, img_grid_vec, similarity_features, lbls = data
    similarity_features = similarity_features.to(torch.float32).to(device)
    img_vecs = img_vecs.to(torch.float32).to(device)
    
    output = None
    for model in models:
        if(output is None):
            temp = model(img_vecs, similarity_features)
            arg_maxes = torch.argmax(nn.functional.softmax(temp, dim=-1), dim=-1)
            output = nn.functional.one_hot(arg_maxes, num_classes=50)
        else:
            temp = model(img_vecs, similarity_features)
            arg_maxes = torch.argmax(nn.functional.softmax(temp, dim=-1), dim=-1)
            output += nn.functional.one_hot(arg_maxes, num_classes=50)
    
    output = output/len(models)

    p_lbls = torch.argmax(nn.functional.softmax(output, dim=-1), dim=-1).tolist()
    t_lbls = lbls.tolist()

    true_lbls += t_lbls
    predicted_lbls += p_lbls

with open('validation_results_private_set.txt', 'w') as f:
    f.write("\n".join(str(item) for item in predicted_lbls))

In [6]:
with open('validation_results_private_set.txt', 'r') as f:
    content = f.read()

print(content)

45
33
16
12
39
23
40
33
0
11
40
14
48
48
38
26
11
11
31
43
14
34
39
46
27
42
46
39
21
45
44
7
44
7
6
34
28
9
6
14
38
29
31
27
38
20
24
28
5
49
46
9
25
31
37
40
33
35
15
26
10
28
25
18
17
16
33
17
2
19
5
15
30
42
46
2
17
22
37
20
8
42
28
5
5
26
17
12
2
3
39
11
36
40
12
41
42
4
36
6
2
26
39
21
23
33
8
33
20
43
21
10
3
8
49
23
22
10
32
42
2
38
44
8
1
30
12
40
38
14
12
13
37
3
22
10
48
37
26
23
11
11
35
13
27
37
26
49
43
33
19
24
13
25
45
20
36
34
29
10
27
33
42
12
8
46
39
6
8
47
8
48
33
6
9
34
39
19
17
27
1
12
17
22
33
41
31
1
42
29
16
39
39
19
29
42
20
28
33
28
24
2
19
1
48
34
14
0
25
34
19
45
28
13
35
16
48
0
16
22
32
46
44
40
1
48
28
43
0
17
27
15
27
20
18
11
38
0
45
47
46
32
14
32
37
40
41
28
7
27
43
11
22
32
36
0
38
33
46
9
35
31
33
40
15
13
22
16
35
32
45
23
29
30
43
15
40
37
23
37
1
46
9
35
7
22
30
37
44
15
29
43
8
4
35
8
38
30
36
19
23
18
7
42
8
4
48
24
34
19
41
13
13
49
20
20
2
1
31
46
38
40
18
47
42
6
1
12
3
0
14
14
22
24
2
39
36
3
3
20
2
43
44
30
6
42
10
23
43
47
11
1
44
25
27
